In [1]:
import spacy
import re
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span

## Using the Entity Ruler object from SpaCy

In [2]:
from spacy.lang.nb import Norwegian
from spacy.pipeline import EntityRuler

In [3]:
patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]

In [4]:
reg = [{"label": "FNR", "pattern":[{"TEXT": {"REGEX": r'(\b\d{11}\b)|(\b\d{6}\s\d{5}\b)'}}]}]

In [5]:
tx="Jeg er Robindra og mitt fødselsnummer er 15044216652"

In [6]:
nlp = spacy.load("nb_core_news_lg")
#nlp = Norwegian() is an empty model
ruler = EntityRuler(nlp)
ruler.add_patterns(reg)
nlp.add_pipe(ruler)
from collections import defaultdict

In [7]:
#Example text
txt = "Selv om det hører inn under sakprosaen, er det mange forfattere som skriver essayistisk; som altså vil si assosiativt og sirkulært i formen. Mange vil kalle essayet en blandingsgenre; Mia Bull Gundersen skriver for eksempel: «Essayet er en utpreget litterær blandingsform og befinner seg et sted mellom fiksjon og sakprosa» (1). En del tekster kan være essayistiske i formen, selv om de faller inn under andre genrebetegnelser. Dette gjelder både artikler, kronikker, petiter, kåserier og romaner. Essayet er en forholdsvis kort tekst, skrevet i førsteperson entall i en ikke-konkluderende form. Jo Bech-Karlsen (2) sier essayet skal være kunnskapsrikt, men ikke lærd i teoretisk vitenskapelig forstand. Ofte stiller essayet flere spørsmål enn det besvarer, og forfatteren av essayet henvender seg gjerne til sin likemann; en kunnskapsrik og opplyst leser. I motsetning til en artikkel, som er bygget opp med en innledning, drøfting og konklusjon, har essayet verken innledning eller avslutning. Leseren får ikke nødvendigvis svar på de spørsmålene teksten stiller, men skal likevel være klokere enn før han eller hun gikk i gang med lesningen. I likhet med essayet selv, omfatter essayets historie flere tradisjoner og avstikkere. De fleste betrakter Michel de Montaigne som essayets far. Montaigne (1533–1592) var en fransk adelsmann, politiker, filosof og forfatter som hadde stor innflytelse i sin samtid. Som trettiåring trakk han seg tilbake til familiens slott i Bordeaux, for i ensomhet å reflektere og skrive, i sitt tårn, omgitt av sine bøker. Begrepet essay er hentet fra tittelen på hans bøker: Les Essais I–III, som kom ut mellom 1780 og 1788. Det franske begrepet kan oversettes med forsøk, som peker tilbake på essayets åpne, prøvende og ikke-konkluderende form. Selv fortsatte Montaigne å redigere og endre på essayene sine resten av livet. Dette forsterker inntrykket av essayet som uferdig og åpent. Det er ofte store og evige spørsmål som drøftes i essayet, samtidig som ingenting synes å være for smått."

In [8]:
doc = nlp(txt)
print([(ent.text, ent.label_, str(ent.start), str(ent.end)) for ent in doc.ents])

[('Mia Bull Gundersen', 'PER', '33', '36'), ('Essayet', 'PER', '41', '42'), ('Essayet', 'PER', '92', '93'), ('Bech-Karlsen', 'ORG', '109', '110'), ('Michel de Montaigne', 'PER', '222', '225'), ('Montaigne', 'PER', '229', '230'), ('Bordeaux', 'LOC', '261', '262'), ('Montaigne', 'PER', '323', '324')]


In [9]:
beams = nlp.entity.beam_parse([doc], beam_width=16, beam_density=0.0001)
for score, ents in nlp.entity.moves.get_beam_parses(beams[0]):
    print (score, ents)
    entity_scores = defaultdict(float)
    for start, end, label in ents:
        # print ("here")
        entity_scores[(start, end, label)] += score
        print ('entity_scores', entity_scores)

for (start, end, label),value in entity_scores.items():
        if label == 'LOCATION':
            print (start, tokens[start], value)

1.0 [(33, 36, 'PER'), (41, 42, 'PER'), (92, 93, 'PER'), (109, 110, 'ORG'), (222, 225, 'PER'), (229, 230, 'PER'), (261, 262, 'LOC'), (323, 324, 'PER')]
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0, (109, 110, 'ORG'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0, (109, 110, 'ORG'): 1.0, (222, 225, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0, (109, 110, 'ORG'): 1.0, (222, 225, 'PER'): 1.0, (229, 230, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 

### Assigning multiple custom entities

In [10]:
from nav_pii_anon.spacy.spacy_model import SpacyModel
from nav_pii_anon.regex_engine import fnr

In [11]:
## Using the Entity Ruler object from SpaCy

from spacy.lang.nb import Norwegian
from spacy.pipeline import EntityRuler

patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]

reg = [{"label": "FNR", "pattern":[{"TEXT": {"REGEX": r'(\b\d{11}\b)|(\b\d{6}\s\d{5}\b)'}}]}]

tx="Jeg er Robindra og mitt fødselsnummer er 15044216652"

nlp = spacy.load("nb_core_news_lg")
#nlp = Norwegian() is an empty model
ruler = EntityRuler(nlp)
ruler.add_patterns(reg)
nlp.add_pipe(ruler)
from collections import defaultdict

#Example text
txt = "Selv om det hører inn under sakprosaen, er det mange forfattere som skriver essayistisk; som altså vil si assosiativt og sirkulært i formen. Mange vil kalle essayet en blandingsgenre; Mia Bull Gundersen skriver for eksempel: «Essayet er en utpreget litterær blandingsform og befinner seg et sted mellom fiksjon og sakprosa» (1). En del tekster kan være essayistiske i formen, selv om de faller inn under andre genrebetegnelser. Dette gjelder både artikler, kronikker, petiter, kåserier og romaner. Essayet er en forholdsvis kort tekst, skrevet i førsteperson entall i en ikke-konkluderende form. Jo Bech-Karlsen (2) sier essayet skal være kunnskapsrikt, men ikke lærd i teoretisk vitenskapelig forstand. Ofte stiller essayet flere spørsmål enn det besvarer, og forfatteren av essayet henvender seg gjerne til sin likemann; en kunnskapsrik og opplyst leser. I motsetning til en artikkel, som er bygget opp med en innledning, drøfting og konklusjon, har essayet verken innledning eller avslutning. Leseren får ikke nødvendigvis svar på de spørsmålene teksten stiller, men skal likevel være klokere enn før han eller hun gikk i gang med lesningen. I likhet med essayet selv, omfatter essayets historie flere tradisjoner og avstikkere. De fleste betrakter Michel de Montaigne som essayets far. Montaigne (1533–1592) var en fransk adelsmann, politiker, filosof og forfatter som hadde stor innflytelse i sin samtid. Som trettiåring trakk han seg tilbake til familiens slott i Bordeaux, for i ensomhet å reflektere og skrive, i sitt tårn, omgitt av sine bøker. Begrepet essay er hentet fra tittelen på hans bøker: Les Essais I–III, som kom ut mellom 1780 og 1788. Det franske begrepet kan oversettes med forsøk, som peker tilbake på essayets åpne, prøvende og ikke-konkluderende form. Selv fortsatte Montaigne å redigere og endre på essayene sine resten av livet. Dette forsterker inntrykket av essayet som uferdig og åpent. Det er ofte store og evige spørsmål som drøftes i essayet, samtidig som ingenting synes å være for smått."

doc = nlp(tx)
print([(ent.text, ent.label_, str(ent.start), str(ent.end)) for ent in doc.ents])

beams = nlp.entity.beam_parse([doc], beam_width=16, beam_density=0.0001)
for score, ents in nlp.entity.moves.get_beam_parses(beams[0]):
    print (score, ents)
    entity_scores = defaultdict(float)
    for start, end, label in ents:
        # print ("here")
        entity_scores[(start, end, label)] += score
        print ('entity_scores', entity_scores)

for (start, end, label),value in entity_scores.items():
        if label == 'LOCATION':
            print (start, tokens[start], value)

### Assigning multiple custom entities

from nav_pii_anon.spacy.spacy_model import SpacyModel

[('Robindra', 'PER', '2', '3'), ('15044216652', 'FNR', '7', '8')]
1.0 [(2, 3, 'PER'), (7, 8, 'FNR')]
entity_scores defaultdict(<class 'float'>, {(2, 3, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(2, 3, 'PER'): 1.0, (7, 8, 'FNR'): 1.0})


In [19]:
dist_checker = fnr.RegexFnr()

dist = dist_checker.validate_pnr('170415443149')

The Levenstein distance from a pnr is 1.0


In [13]:
str(doc.ents[1])

'15044216652'

In [14]:
dist

In [15]:
import re 
re.match(r"(0[1-9]|[12][0-9]|3[01])(0[1-9]|1[012])", )

TypeError: match() missing 1 required positional argument: 'string'